In [1]:
import xml.etree.ElementTree as ET
import requests
import os
import collections
import time

In [2]:
SLEEP_BEFORE_QUEUE = 10  # Wait for 10s before checking if the request is ready

In [3]:
user = 'eekspider'
# user = '2624johnm'
# user = 'abehk'

In [4]:
def request_collection_of_user(user, queued_users):
    r = requests.get(f'https://boardgamegeek.com/xmlapi/collection/{user}')
    if r.status_code == 200:
        # Success! Store the file
        with open(fname, 'w') as f:
            f.write(r.text)
    elif r.status_code == 202:
        # Request is in the queue
        queued_users.append(user)

In [5]:
fname = f'./raw_data/collection/{user}.xml'
queued_users = []

if os.path.isfile(fname):
    # No need to call the API again
    with open(fname, 'r') as f:
        r_text = f.read()
else:
    request_collection_of_user(user, queued_users)

In [6]:
print(f"Requests in queue: {len(queued_users)}")

Requests in queue: 0


In [7]:
failed_users = []

if queued_users:
    time.sleep(SLEEP_BEFORE_QUEUE)
    for user in queued_users:
        request_collection_of_user(user, failed_users)

In [8]:
assert not failed_users  # We shouldn't have any failed requests (happy path)

## Parsing the XML

In [9]:
xml_file = f'./raw_data/collection/{user}.xml'

with open(xml_file, 'r') as f:
    r_text = f.read()
    root = ET.fromstring(r_text)

In [10]:
for bg in root:
    bg_name = bg[0].text
    coll_id = bg.attrib['collid']  # I don't really know what this is, but I guess it is the id of this instance of the boardgame in the list
    object_id = bg.attrib['objectid']  # This is the boardgame identifier

    for field in bg:
        if field.tag == 'stats':
            rating_val = field[0].attrib['value']
        if field.tag == 'yearpublished':
            year_published = field.text
            # break

    print(bg_name, rating_val, year_published, coll_id, object_id)

Architekton N/A 2005 7044466 16499
Australian Rails N/A 1994 7044566 935
Big Boggle N/A 1979 7276794 10550
Boggle Bowl N/A 1987 7276793 2495
British Rails N/A 1984 7044579 2689
Carcassonne N/A 2000 7276771 822
Carcassonne: Expansion 1 – Inns & Cathedrals N/A 2002 7276774 2993
Carcassonne: Expansion 2 – Traders & Builders N/A 2003 7276775 5405
Carcassonne: Expansion 3 – The Princess & The Dragon N/A 2005 7276780 15158
Carcassonne: Expansion 4 – The Tower N/A 2006 7276776 21385
Carcassonne: Expansion 5 – Abbey & Mayor N/A 2007 7276778 31784
Carcassonne: GQ Promo Tiles N/A 2006 7271479 26615
Carcassonne: Hunters and Gatherers N/A 2002 7276765 4390
Carcassonne: King & Scout N/A 2003 7276781 7707
Carcassonne: The Castle N/A 2003 7276768 7717
Carcassonne: The Count of Carcassonne N/A 2004 7276769 12903
Carcassonne: The River N/A 2001 7278118 2591
Carcassonne: The River II N/A 2005 7276770 18836
CATAN N/A 1995 7049980 13
CATAN N/A 1995 7276748 13
Catan Card Game N/A 1996 7271474 278
CATAN: Ci

# Comments

- It is possible to have two repeated games in a user's list.
    - Check `eekspider`'s `CATAN` game.